# (2)疑似ラベル

### 1.データの作成

In [1]:
#データ読み込み
import pandas as pd
import numpy as np
df=pd.read_excel('C:/Users/Administrator/python/kaggle/driver_detection/original_data/pursuade_list_dump.xlsx')
df_val=pd.read_excel('C:/Users/Administrator/python/kaggle/driver_detection/original_data/val_list.xlsx')

df['class']=df['class'].astype('str')

#重みを作成
df['weight']=1e-8

#教師ありデータの重みは1
df.loc[df['label']=='label','weight']=1

In [2]:
df.loc[df['label']=='unlabel','classname']='c'+df.loc[df['label']=='unlabel','class']

In [3]:
#データの確認
display(df.head(5))
display(df.tail(5))

,subject,pass,classname,label,class,weight
0,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0
1,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0
2,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0
3,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0
4,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0


,subject,pass,classname,label,class,weight
100187,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c1,unlabel,1,1.000000e-08
100188,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c6,unlabel,6,1.000000e-08
100189,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c0,unlabel,0,1.000000e-08
100190,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c8,unlabel,8,1.000000e-08
100191,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c6,unlabel,6,1.000000e-08


In [4]:
#教師なしデータのみ抽出
df_unlabel=df.copy()
df_unlabel=df_unlabel[df_unlabel['label']=='unlabel']
df_unlabel.head()

,subject,pass,classname,label,class,weight
20466,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c8,unlabel,8,1.000000e-08
20467,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c6,unlabel,6,1.000000e-08
20468,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c3,unlabel,3,1.000000e-08
20469,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c5,unlabel,5,1.000000e-08
20470,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c7,unlabel,7,1.000000e-08


In [5]:
from keras import layers
from keras import models
import numpy as np

Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\Tersorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\Tersorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\Tersorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\Te

In [6]:
#データGeneratorを作成
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=5*2,
                                  width_shift_range=0.1*2,
                                  height_shift_range=0.1*2,
                                  shear_range=0.1*2,
                                  zoom_range=0.1*2,
                                  horizontal_flip=False,
                                  fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)
unlabel_predict_datagen = ImageDataGenerator(rescale=1./255)

unlabel_predict_generator=unlabel_predict_datagen.flow_from_dataframe(
    df_unlabel, 
    x_col='pass',
    y_col=None,
    class_mode=None,
    target_size=(200,200),
    batch_size=20,
    shuffle=False)

Found 79726 validated image filenames.


In [7]:
#validation_dirの場所を指定
validation_dir='C:/Users/Administrator/python/kaggle/driver_detection/devide_data/validation'

validation_generator=test_datagen.flow_from_directory(
validation_dir,
target_size=(200,200),
batch_size=50,
class_mode='categorical')

Found 1958 images belonging to 10 classes.


### 2.モデルの読み込み

In [8]:
#最良だったModelを読み込み
from keras.models import load_model
model=load_model('model0913-VGG16-fine1-07-0.25(0.45003).h5')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
#転移学習するために、block3_conv1より前のパラメータを凍結
model.trainable=True

set_trainable=False
for layer in model.layers:
    if layer.name=='block3_conv1':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False

In [10]:
from keras import models, optimizers, layers
#optimizerを指定
model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-6),metrics=['acc'])

In [11]:
#trainable paramsを確認
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0   

### 3.疑似ラベルによる学習

In [12]:
from keras.callbacks import ModelCheckpoint
t1=30
alpha=3
learning_history=[]

for epoch_n in range(1,50):
    print('予測値を更新します...')
    predict = model.predict_generator(unlabel_predict_generator, steps=np.ceil(79726/20))        
    df.loc[df['label']=='unlabel','class']=np.argmax(predict,axis=1)
    df.loc[df['label']=='unlabel','classname']='c'+df.loc[df['label']=='unlabel','class'].astype('str')
    display(df.head(5))
    display(df.tail(5))
    
    #学習データの重みを変更
    #t1-t2は徐々に増加させ、最終的にunlabelの重みがalphaになる
    if epoch_n<=t1:
        df.loc[df['label']=='unlabel','weight']=alpha*(epoch_n-0)/(t1)
        print('epoch<t1より、重みを更新します...。')
        display(df.head(5))
        display(df.tail(5))
        
    #重みはalphaに固定
    if epoch_n>=t1:
        df.loc[df['label']=='unlabel','weight']=alpha
        print('epoch>t1より、重みを更新します...。')
        display(df.head(5))
        display(df.tail(5))
    
    print('epoch:{}の計算を開始'.format(epoch_n))
    train_generator=train_datagen.flow_from_dataframe(
         df, 
         x_col='pass',
         y_col='classname',
         class_mode='categorical',
         target_size=(200,200),
         batch_size=50,
         weight_col='weight',
         shuffle=False)

    checkpoint = ModelCheckpoint(
         filepath="model0913-VGG16-persuade-{val_loss:.4f}.h5",
         monitor='val_loss',
         save_best_only=True,
         period=1,
         )
    
    history = model.fit_generator(
         train_generator,
         steps_per_epoch=434,
         epochs=1,
         validation_data=validation_generator,
         validation_steps=41,
         callbacks=[checkpoint]
         )
    
    learning_history.append(history.history)

予測値を更新します...


,subject,pass,classname,label,class,weight
0,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0
1,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0
2,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0
3,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0
4,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0


,subject,pass,classname,label,class,weight
100187,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c1,unlabel,1,1.000000e-08
100188,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c3,unlabel,3,1.000000e-08
100189,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c4,unlabel,4,1.000000e-08
100190,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c6,unlabel,6,1.000000e-08
100191,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c5,unlabel,5,1.000000e-08


epoch<t1より、重みを更新します...。


,subject,pass,classname,label,class,weight
0,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0
1,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0
2,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0
3,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0
4,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0


...

,subject,pass,classname,label,class,weight
100187,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c1,unlabel,1,3.0
100188,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c3,unlabel,3,3.0
100189,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c4,unlabel,4,3.0
100190,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c6,unlabel,6,3.0
100191,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c5,unlabel,5,3.0


epoch>t1より、重みを更新します...。


,subject,pass,classname,label,class,weight
0,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0
1,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0
2,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0
3,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0
4,p012,C:/Users/Administrator/python/kaggle/driver_de...,c0,label,0,1.0


,subject,pass,classname,label,class,weight
100187,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c1,unlabel,1,3.0
100188,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c3,unlabel,3,3.0
100189,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c4,unlabel,4,3.0
100190,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c6,unlabel,6,3.0
100191,NaN,C:/Users/Administrator/python/kaggle/driver_de...,c5,unlabel,5,3.0


epoch:49の計算を開始
Found 100192 validated image filenames belonging to 10 classes.
Epoch 1/1
434/434 [==============================] - 295s 680ms/step - loss: 0.7084 - acc: 0.9119 - val_loss: 0.5527 - val_acc: 0.9183
